# import library

In [ ]:
# Import library
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore')

# load csv file

In [ ]:
# Load a CSV file containing radiology reports
# Ensure the CSV file includes a column named "text" that contains the radiology report texts
df = pd.read_csv('../data/reports.csv')

# model preparation

In [ ]:
# download the model from Hugging Face Models
model_name = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto",)

# execute information extraction from CT reports

In [ ]:
# Define the instruction and system markers used for formatting the prompts
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# Define the default system prompt which describes the task clearly to the model
DEFAULT_SYSTEM_PROMPT = '''以下は肺癌について書かれた報告です。
腫瘍の大きさ、陰影の種類、腫瘍の場所、リンパ節腫大の部位（ない場合は"なし"）、転移の部位（ない場合は"なし"）を端的に抜き出して下さい。
回答形式は以下のjson形式として、必ずjsonだけを返してください。
keyとvalueは必ずダブルクオートで囲んでください。

{
"大きさ": ???,
"陰影の種類": ???,
"腫瘍の場所": ???,
"リンパ節腫大": ???,
"転移": ???
}
'''

# Create an empty list to store the processed JSON outputs
text_pp = []

# Iterate over each text entry in the dataframe's "text" column
for i, text in enumerate(df["text"].values):
    # Remove newlines from the text for cleaner formatting
    text = text.replace("\n", "")
    
    # Prefix the text with a label indicating that it is a report
    text = "報告: \n" + text

    # Construct the full prompt by embedding the system and task instructions, followed by the specific text
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ### json: ".format(
        bos_token=tokenizer.bos_token,  # BOS token (beginning of sentence token) specific to the tokenizer
        b_inst=B_INST,  # Instruction start marker
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",  # Include the system prompt in the proper format
        prompt=text,  # Append the current text (report)
        e_inst=E_INST,  # Instruction end marker
    )

    # Generate the output JSON response using the model
    with torch.no_grad():  # Disable gradient computation for inference
        token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
        # Encode the prompt into token IDs for input to the model

        output_ids = model.generate(
            token_ids.to(model.device),  # Move token IDs to the device (e.g., GPU) where the model is loaded
            max_new_tokens=320,  # Limit the maximum number of tokens to generate
            pad_token_id=tokenizer.pad_token_id,  # Specify padding token ID
            eos_token_id=tokenizer.eos_token_id,  # Specify end-of-sequence token ID
        )

    # Decode the generated output back into text, skipping special tokens
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

    # Print the text and its generated output for debugging or logging purposes
    print(f"text num{i}")
    print(text)
    print(output)
    print()

    # Append the generated JSON output to the list
    text_pp.append(output)

# Add the generated JSON responses as a new column in the dataframe
df["text_json"] = text_pp

# Save the result
df.to_csv("./llama2_ver1009_extract.csv", index=False)

text num0
報告: 
左上葉に径18mm 大のSSN を認めます。AAH やAIS の可能性があります。右下葉にもGGN を散見します。炎症性変化かもしれませんが、フォローにて変化をご確認ください。左下葉に線状索状影を認め陳旧性炎症性変化が疑われます。縦隔や肺門に有意なリンパ節腫大は指摘できません。胸水はありません。

{
"大きさ": "18mm",
"陰影の種類": "SSN、AISの可能性あり",
"腫瘍の場所": "左上葉",
"リンパ節腫大": "なし",
"転移": "なし"
}

text num1
報告: 
肺野背景に軽度の気腫性変化を認めます。肺尖部にはブラを認めます。左肺上葉にやや分葉状を呈する、境界明瞭なすりガラス状陰影を認めます。中心には小さい高吸収域を認めます。すりガラス影の⻑径は18mm です。高分化腺癌を疑う所見ですが、サイズ変化をみるために、まずは1,2 ヶ月程度でのCT 再検をお願いします。肺癌とした場合、T1 と考えます。同病変は胸膜とは離れています。肺癌とした場合、積極的に肺転移を疑う病変は指摘できません。また右肺下葉にもすりガラス影が散見されます。これらは境界不明瞭であり、腫瘍性病変よりも炎症性変化を疑います。上記病変と併せてサイズフォローをお願いします。両肺下葉背側には陳旧性炎症性変化と思われる、矩形の小結節を認めます。左肺下葉には索状影を認めます。陳旧性炎症性変化と考えます。縦隔・肺門リンパ節に病的腫大は認めません。胸水貯留はありません。食道裂孔ヘルニアがあります。副腎に転移を疑う結節は指摘できません。軟部条件がありませんので、肝転移の有無は評価困難です。

{
"大きさ": "18mm",
"陰影の種類": "すりガラス状",
"腫瘍の場所": "左肺上葉",
"リンパ節腫大": "なし",
"転移": "なし"
}

text num2
報告: 
1 年前のCT と比較しました。左肺上区S1+2 に⻑径18mm 大のpart solid GGN は内部の充実部分は前回よりも減少しているものの、すりガラス影のサイズは前回と比べ縮小なく、僅かに胸膜陥入像を伴っています。経過も併せ、上皮内腺癌(AIS)、微小浸潤腺癌(MIA) などの腫瘍性病変を考える所見です。両肺尖部にbulla、右肺上葉・中葉や左肺下葉に小結節、